# 1. Mathematical Justification
Cross Entropy Cost Function:
$$
C = -\sum_j{t_j log(y_j)}
$$

Softmax:
$$
y_i = \frac{e^{z_i}}{\sum_{j \in group }{e^{z_{j}}}}
$$

Log Probability:
$$
log(y_i) 
= log(\frac{e^{z_i}}{\sum_{j \in group }{e^{z_{j}}}})
= log(e^{z_i}) - log(\sum_{j \in group }{e^{z_{j}}})
$$

calculating the above terms could be dangerous, in a way that the exponential of big numbers can be really big, which is not a best practise for any programming languages. 
Assuming we have N elements in our group. $$ j \in (1,2,...,N)$$ instead of calculating potentially N giant numbers and add them up, we try to calculate one giant number one time - the biggest of the group, and then calculate N small numbers.

$$
log(\sum_{j \in group }{e^{z_{j}}})
= log(e^{z_{max}}\sum_{j \in group }{(e^{z_{j}} / e^{z_{max}})})
= log(e^{z_{max}}\sum_{j \in group }{e^{z_{j} - z_{max}}})
$$

For those of you who prefer this format:

$$
log(e^{z_{1}} + e^{z_{2}} + ... + e^{z_{N}})
= log(\frac{(e^{z_{1}} + e^{z_{2}} + ... + e^{z_{N}}) }{e^{z_{max}}} e^{z_{max}} ) 
= log((e^{z_{1}-z_{max}} + e^{z_{2}-z_{max}} + ... + e^{z_{N}-z_{max}})*e^{z_{max}} ) 
$$

Since $$ log(ab) = log(a) + log(b)$$

So that:
$$
log(e^{z_{1}} + e^{z_{2}} + ... + e^{z_{N}})
= log(e^{z_{1}-z_{max}} + e^{z_{2}-z_{max}} + ... + e^{z_{N}-z_{max}}) + log(e^{z_{max}}) 
= log(e^{z_{1}-z_{max}} + e^{z_{2}-z_{max}} + ... + e^{z_{N}-z_{max}}) + z_{max}
$$

This is basically the mathematically representation of the more "numerically stable" implementation. 
since $ z_i - z_{max} $ are all negative, so there won't be any exponential calculation that can be greater than 1. So in this way, we almost ** avoid large exponential calculation ** out of our implementation.

Below is the Octave correspondence of the math representation above:

$$ \mbox{ret = log(sum(exp(a - maxs_big), 1)) + maxs_small}$$

# 2. Practical Justification

In [70]:
# Here is the Octave implementation
# 
# function ret = log_sum_exp_over_rows(a)
#   % This computes log(sum(exp(a), 1)) in a numerically stable way
#   maxs_small = max(a, [], 1);
#   maxs_big = repmat(maxs_small, [size(a, 1), 1]);
#   ret = log(sum(exp(a - maxs_big), 1)) + maxs_small;
# end

In [71]:
import numpy as np
import numpy.matlib

In [72]:
z = np.arange(12).reshape(4,3)
print z

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]


In [73]:
# Numeric Unstable Implementation
np.log(np.sum(np.exp(z), axis=0))

array([  9.05106304,  10.05106304,  11.05106304])

In [74]:
# Numeric Stable Implementation
maxs_small = z.sum(axis=0)
print 'maxs_small:'
print maxs_small
maxs_big = np.matlib.repmat(maxs_small, np.size(z, 0), 1)
print 'maxs_big:'
print maxs_big
ret = np.log(np.sum(np.exp(z - maxs_big), axis=0)) + maxs_small
print 'ret:'
print ret

maxs_small:
[18 22 26]
maxs_big:
[[18 22 26]
 [18 22 26]
 [18 22 26]
 [18 22 26]]
ret:
[  9.05106304  10.05106304  11.05106304]
